# Convolutional Neural Network for Tweets Classification (`racist` and `non-racist`)

Convolutional Neural Network (CNNs) was originally for Computer Vision tasks. In our project, we will implement CNN for tweets classification task. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install -U torch

     |████████████████████████████████| 752.0MB 16kB/s 
ERROR: torchvision 0.5.0 has requirement torch==1.4.0, but you'll have torch 1.5.0 which is incompatible.
  Found existing installation: torch 1.4.0
    Uninstalling torch-1.4.0:
      Successfully uninstalled torch-1.4.0


In [3]:
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


## Import required Python libraries

In [0]:
import torch
import torchtext
from torchtext.data import Field, LabelField
from torchtext.data import TabularDataset
from torchtext.data import Iterator, BucketIterator
import spacy
import en_core_web_sm
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
from tqdm import tqdm, trange
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix

In [5]:
## Set seed of randomization and working device
manual_seed = 77
torch.manual_seed(manual_seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
n_gpu = torch.cuda.device_count()
if n_gpu > 0:
    torch.cuda.manual_seed(manual_seed)

cpu


## Load dataset

In this model, our initial collection of tweets involved tweets from two sources. The first tweet set was provided by the course teaching staff, comprising over 76,600 tweets stored in a .txt file. The second tweet set was collected by the project team members by scraping tweets related to the hashtag #ChinaLiedAndPeopleDied. This hashtag was deemed one of the hashtags that would yield a reasonable number of negative and potentially racist tweets. In the early weeks of this project, approximately 25,000 tweets were collected and stored in a csv file.

We have already preprocessed (tokenization, removing URLs, mentions, hashtags and so on) the tweets and placed it in three files as ``train.tsv``, ``dev.tsv`` and ``test.tsv``. 

Tokenizer

In [0]:
spacy_en = en_core_web_sm.load()
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

`Fileds`

In [0]:
TEXT = Field(sequential=True, tokenize=tokenize_en, lower=True)
LABEL = Field(sequential=False, unk_token = None)

Load dataset using `TabularDataset`

In [0]:
train, val, test = TabularDataset.splits(
               path="./drive/My Drive/Colab Notebooks/trends/", 
               train='train.tsv', validation="dev.tsv", test="test.tsv", 
               format='tsv',
               skip_header=True, 
               fields=[('tweet', TEXT), ('label', LABEL)])

Build our vocabulary to map words and labels to integers. 

In [0]:
TEXT.build_vocab(train, min_freq=2)
LABEL.build_vocab(train)

In [10]:
print("Vocabulary size of TEXT:",len(TEXT.vocab.stoi))
print("Vocabulary size of LABEL:",len(LABEL.vocab.stoi))

Vocabulary size of TEXT: 1336
Vocabulary size of LABEL: 2


In [11]:
LABEL.vocab.stoi

defaultdict(<function torchtext.vocab._default_unk_index>,
            {'non-racist': 0, 'racist': 1})

Here we construct the Iterators to get the train, dev, and test splits. We use `BucketIterator` to initialize the Iterators for the train, dev, and test data.

In [0]:
train_iter, val_iter, test_iter = BucketIterator.splits(
 (train, val, test), 
 batch_sizes=(64,256,256),
 sort_key=lambda x: len(x.tweet), 
 sort=True,
 sort_within_batch=True
)

In [13]:
for batch in train_iter:
    tweets = batch.tweet
    labels = batch.label
    break  
    
print('tweets:', tweets.shape)
print('labels:', labels.shape)


tweets: torch.Size([12, 64])
labels: torch.Size([64])


## Let's build a CNN model for tweets racism classification task!

#### First, we need to create a input tensor and the label tensor. 

In [0]:
input_sentence = 'I like this movie very much!'

In [15]:
text_token = TEXT.preprocess(input_sentence)
print("text_token:", text_token)
input_tensor = TEXT.process([text_token])
print(input_tensor)
print("shape of input_tensor: ", input_tensor.shape)

text_token: ['i', 'like', 'this', 'movie', 'very', 'much', '!']
tensor([[ 48],
        [150],
        [ 19],
        [  0],
        [179],
        [213],
        [ 15]])
shape of input_tensor:  torch.Size([7, 1])


In [16]:
label_token = LABEL.preprocess('non-racist')
print("label_token:", label_token)
label_tensor = LABEL.process([label_token])
print("shape of label_tensor: ", label_tensor.shape)

label_token: non-racist
shape of label_tensor:  torch.Size([1])


In [17]:
VOCAB_SIZE = len(TEXT.vocab.stoi)
print("VOCAB_SIZE:",VOCAB_SIZE)
embedding = nn.Embedding(VOCAB_SIZE, 5)

VOCAB_SIZE: 1483


In [18]:
#feed the tensors of our sample batch to the embedding module 
print("input_tensor: ", input_tensor.shape)
input_embeddings = embedding(input_tensor)
print("tweet input word embeddings size: ", input_embeddings.size()) 

input_tensor:  torch.Size([7, 1])
tweet input word embeddings size:  torch.Size([7, 1, 5])


In [19]:
input_embeddings = input_embeddings.permute(1,0,2)
print(input_embeddings.shape)
input_embeddings = input_embeddings.unsqueeze(1)
print(input_embeddings.shape)

torch.Size([1, 7, 5])
torch.Size([1, 1, 7, 5])


In [20]:
convolute_region2 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=(2,5), stride=1)
convolute_region2_output = convolute_region2(input_embeddings)
print(convolute_region2_output.shape)

torch.Size([1, 2, 6, 1])


In [21]:
ReLU_fn = nn.ReLU()
convolute_region2_output = ReLU_fn(convolute_region2_output)
print("convolute_region2_output:", convolute_region2_output.shape)

convolute_region2_output: torch.Size([1, 2, 6, 1])


In [22]:
convolute_region2_output = convolute_region2_output.squeeze(-1)
print("convolute_region2_output:", convolute_region2_output.shape)

convolute_region2_output: torch.Size([1, 2, 6])


In [23]:
convolute_region3 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=(3,5), stride=1)
convolute_region3_output = convolute_region3(input_embeddings)
convolute_region3_output = ReLU_fn(convolute_region3_output)
convolute_region3_output = convolute_region3_output.squeeze(-1)
print(convolute_region3_output.shape)

torch.Size([1, 2, 5])


In [24]:
convolute_region4 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=(4,5), stride=1)
convolute_region4_output = convolute_region4(input_embeddings)
convolute_region4_output = ReLU_fn(convolute_region4_output)
convolute_region4_output = convolute_region4_output.squeeze(-1)
print(convolute_region4_output.shape)

torch.Size([1, 2, 4])


In [0]:
kernel_sizes = [2,3,4] #three region size
kernel_num = 2      # number of kernels 
embedding_dim = 5   # word embedding size
Ci = 1              # number of input channel

convolution_layers = nn.ModuleList([
    nn.Conv2d(in_channels = Ci, out_channels = kernel_num, kernel_size = (K, embedding_dim)) 
                                    for K in kernel_sizes  ])

convolute_outputs = [F.relu(conv(input_embeddings)).squeeze(3) for conv in convolution_layers]  

In [26]:

for i, item in enumerate(convolute_outputs):
    print(i," shape: ",item.shape)


0  shape:  torch.Size([1, 2, 6])
1  shape:  torch.Size([1, 2, 5])
2  shape:  torch.Size([1, 2, 4])


#### Max Pooling layers

In [27]:
maxpool_region2 = F.max_pool1d(convolute_region2_output, convolute_region2_output.size(2))
print(convolute_region2_output)
print(maxpool_region2)
print(maxpool_region2.shape) 

tensor([[[0.7373, 0.1499, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1866, 0.0000, 1.0698, 0.0000, 0.0000, 0.2386]]],
       grad_fn=<SqueezeBackward1>)
tensor([[[0.7373],
         [1.0698]]], grad_fn=<SqueezeBackward1>)
torch.Size([1, 2, 1])


In [28]:
maxpool_region2 = maxpool_region2.squeeze(-1)
print(maxpool_region2.shape)

torch.Size([1, 2])


In [29]:
maxpool_region3 = F.max_pool1d(convolute_region3_output, convolute_region3_output.size(2))
maxpool_region3 = maxpool_region3.squeeze(-1)
print(maxpool_region3.shape)

torch.Size([1, 2])


In [30]:
maxpool_region4 = F.max_pool1d(convolute_region4_output, convolute_region4_output.size(2))
maxpool_region4 = maxpool_region4.squeeze(-1)
print(maxpool_region4.shape)

torch.Size([1, 2])


In [0]:
# use for-loop to perform max-pooling on convolute_outputs 
max_pooling_outputs = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in convolute_outputs]  

In [32]:
for i, item in enumerate(max_pooling_outputs):
    print(i," shape: ",item.shape) 

0  shape:  torch.Size([1, 2])
1  shape:  torch.Size([1, 2])
2  shape:  torch.Size([1, 2])


In [33]:
# concatenate the 6 features together to form a feature vector.
concat = torch.cat((maxpool_region2, maxpool_region3, maxpool_region4), dim = -1)
print(concat.shape) # [batch size, feature size]

torch.Size([1, 6])


In [34]:
concat_list = torch.cat(max_pooling_outputs, 1)
print(concat_list.shape) # [batch size, feature size]

torch.Size([1, 6])


In [0]:
drop_out = nn.Dropout(p=0.5)

In [128]:
drop_output = drop_out(concat)
print("drop_output:", drop_output.shape)

drop_output: torch.Size([1, 6])


In [129]:
fc = nn.Linear(3 * 2 , 2)  
fc_output = F.softmax(fc(drop_output),dim=1)
print("fc_output:", fc_output.shape)
print(fc_output)

fc_output: torch.Size([1, 2])
tensor([[0.3726, 0.6274]], grad_fn=<SoftmaxBackward>)


In [130]:
# We use nn.CrossEntropyLoss() as our loss function. 
weights = [1, 9]
class_weights = torch.FloatTensor(weights)
criterion = nn.CrossEntropyLoss(weight=class_weights)
criterion(fc_output, label_tensor)

tensor(0.8286, grad_fn=<NllLossBackward>)

Till now, we have implemented the sample CNN successfully. 


## Build a `class` for CNN of text classification

In [0]:
class CNN_Text(nn.Module):
    def __init__(self, vocabulary_size, embedding_dim, output_size, kernel_num, region_sizes, dropout):
        '''
        vocabulary_size: vocabulary size
        embedding_dim: word embedding size
        output_size: number of classes in prediction
        kernel_num: number of kernels (number of output channels of convolutional layers)
        region_sizes: height of kernels of convolutional layers
        dropout: dropout rate
        '''
        super(CNN_Text, self).__init__()
        Ci = 1
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size, embedding_dim = embedding_dim )
        self.convolution_layers = nn.ModuleList([nn.Conv2d(in_channels = Ci, out_channels = kernel_num, kernel_size = (K, embedding_dim)) for K in region_sizes])
        self.dropout = nn.Dropout(dropout) 
        self.fc = nn.Linear(len(kernel_sizes) * kernel_num, output_size)

    def forward(self, x):
        input_embeddings = self.embeddings(x)  
        input_embeddings = input_embeddings.permute(1,0,2)
        input_embeddings = input_embeddings.unsqueeze(1)
        convolute_outputs = [F.relu(conv(input_embeddings)).squeeze(3) for conv in self.convolution_layers]  
        max_pooling_outputs = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in convolute_outputs]         
        concat_list = torch.cat(max_pooling_outputs, 1) # concatenate representations        
        drop_output = self.dropout(concat_list)  # add drop layer    
        fc1_output = self.fc(drop_output)  # get the fc1 using a fully connected layer        
        final_output = F.softmax(fc1_output,dim=1)        
        return final_output


In [0]:
# Hyper Parameters
vocabulary_size = len(TEXT.vocab.stoi) 
embedding_dim = 300 
kernel_sizes = [2,3,4] 
kernels_num = 32  
dropout = 0.5
output_size = 2
lr = 0.01     
num_epoch = 5
model = CNN_Text(vocabulary_size, embedding_dim, output_size, kernels_num, kernel_sizes, dropout)


In [133]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.1)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

CNN_Text(
  (embeddings): Embedding(5639, 300)
  (convolution_layers): ModuleList(
    (0): Conv2d(1, 32, kernel_size=(2, 300), stride=(1, 1))
    (1): Conv2d(1, 32, kernel_size=(3, 300), stride=(1, 1))
    (2): Conv2d(1, 32, kernel_size=(4, 300), stride=(1, 1))
  )
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=96, out_features=2, bias=True)
)

In [134]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,778,390 trainable parameters


In [0]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)   # for backpropagation
loss_func = nn.CrossEntropyLoss(weight=class_weights)  

`train()` and `evaluate()` functions

In [0]:
def train(model, iterator, optimizer, criterion):
    
    model.train()
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        batch_input, labels = batch.tweet, batch.label
        #print(labels)
        batch_input = batch_input
        labels = labels

        optimizer.zero_grad()
        
        outputs = model(batch_input)

        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.cpu().item()

    return epoch_loss / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    all_pred=[]
    all_label = []
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            batch_input, labels = batch.tweet, batch.label
            batch_input = batch_input
            labels = labels

            optimizer.zero_grad()

            outputs = model(batch_input)

            loss = criterion(outputs, labels)

            epoch_loss += loss.cpu().item()
            probabilities, predicted = torch.max(outputs.cpu().data, 1)
            all_pred.extend(predicted)
            all_label.extend(labels.cpu())
    accuracy = accuracy_score(all_label, all_pred)
    f1score = f1_score(all_label, all_pred, average='macro') 
    return epoch_loss / len(iterator), accuracy, f1score

In [138]:
# Train the model
MAX_EPOCH = 5
total_step = len(train_iter)
loss_list = []
acc_list = []

for epoch in trange(MAX_EPOCH, desc="Epoch"):
    train_loss = train(model, train_iter, optimizer, criterion)  
    val_loss, val_acc, val_f1 = evaluate(model, val_iter, criterion)
    state_dict_model = model.state_dict() 
    state = {
        'epoch': epoch,
        'state_dict': state_dict_model,
        'optimizer': optimizer.state_dict()
        }

    torch.save(state, "./drive/My Drive/Colab Notebooks/ckpt_cnn/CNN_TEXT_"+str(epoch+1)+".pt")

    print('\n Epoch [{}/{}], Train Loss: {:.4f}, Validation Loss: {:.4f}, Validation Accuracy: {:.4f}, Validation F1: {:.4f}'.format(epoch+1, MAX_EPOCH, train_loss, val_loss, val_acc, val_f1))
    

Epoch:  20%|██        | 1/5 [00:01<00:05,  1.28s/it]


 Epoch [1/5], Train Loss: 0.6731, Validation Loss: 0.6315, Validation Accuracy: 0.4227, Validation F1: 0.3975


Epoch:  40%|████      | 2/5 [00:02<00:03,  1.28s/it]


 Epoch [2/5], Train Loss: 0.5664, Validation Loss: 0.7922, Validation Accuracy: 0.8682, Validation F1: 0.4647


Epoch:  60%|██████    | 3/5 [00:03<00:02,  1.28s/it]


 Epoch [3/5], Train Loss: 0.4451, Validation Loss: 0.8023, Validation Accuracy: 0.8455, Validation F1: 0.4581


Epoch:  80%|████████  | 4/5 [00:05<00:01,  1.27s/it]


 Epoch [4/5], Train Loss: 0.3810, Validation Loss: 0.6663, Validation Accuracy: 0.7727, Validation F1: 0.5939


Epoch: 100%|██████████| 5/5 [00:06<00:00,  1.28s/it]


 Epoch [5/5], Train Loss: 0.3505, Validation Loss: 0.7073, Validation Accuracy: 0.8682, Validation F1: 0.6411


In [0]:
# Let's see what the results are by creating a predict function
def predict(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    all_pred=[]
    all_label = []
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            batch_input, labels = batch.tweet, batch.label
            batch_input = batch_input
            labels = labels

            optimizer.zero_grad()

            outputs = model(batch_input)

            # identify the predicted class for each example in the batch
            probabilities, predicted = torch.max(outputs.cpu().data, 1)
            # put all the true labels and predictions to two lists
            for pred in predicted:
              if pred.item() == 1:
                all_pred.append('racist')
              else:
                all_pred.append('non-racist')
            
    #print(all_pred, all_label)
    return all_pred


In [100]:
predict(model, test_iter, criterion)

['non-racist',
 'non-racist',
 'non-racist',
 'non-racist',
 'racist',
 'non-racist',
 'non-racist',
 'non-racist',
 'non-racist',
 'non-racist',
 'non-racist',
 'racist',
 'non-racist',
 'non-racist',
 'racist',
 'non-racist',
 'non-racist',
 'non-racist',
 'non-racist',
 'racist',
 'non-racist',
 'non-racist',
 'non-racist',
 'non-racist',
 'racist',
 'racist',
 'racist',
 'non-racist',
 'non-racist',
 'racist',
 'non-racist',
 'non-racist',
 'non-racist',
 'non-racist',
 'racist',
 'racist',
 'non-racist',
 'non-racist',
 'racist',
 'racist',
 'non-racist',
 'non-racist',
 'non-racist',
 'non-racist',
 'non-racist',
 'racist',
 'non-racist',
 'non-racist',
 'racist',
 'non-racist',
 'racist',
 'non-racist',
 'racist',
 'non-racist',
 'non-racist',
 'racist',
 'racist',
 'racist',
 'non-racist',
 'non-racist',
 'non-racist',
 'non-racist',
 'racist',
 'non-racist',
 'racist',
 'racist',
 'non-racist',
 'non-racist',
 'racist',
 'racist',
 'racist',
 'racist']

### References
* http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/
* https://medium.com/jatana/report-on-text-classification-using-cnn-rnn-han-f0e887214d5f
* http://www.davidsbatista.net/blog/2018/03/31/SentenceClassificationConvNets/
* https://github.ubc.ca/MDS-CL-2019-20/COLX_585_trends_students/blob/master/labs/Lab1/cnn_text.ipynb
* https://github.ubc.ca/MDS-CL-2019-20/COLX_585_trends_students/blob/master/labs/Lab2/bert_pytorch.ipynb